In [2]:
import os
import requests
import boto3
import pandas as pd

In [3]:
url = 'https://api.ebird.org/v2/data/obs/US/recent'
key = os.environ['EBIRD_API_KEY']
header = {'X-eBirdApiToken': key}
params = {'back': 30}
response = requests.get(url, headers = header, params=params)


In [23]:
response

<Response [200]>

In [66]:
data = response.json()
display(data)
len(data)

[{'speciesCode': 'daejun',
  'comName': 'Dark-eyed Junco',
  'sciName': 'Junco hyemalis',
  'locId': 'L28786071',
  'locName': 'Home',
  'obsDt': '2024-01-19 17:56',
  'howMany': 1,
  'lat': 42.601049,
  'lng': -82.870429,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S159306976'},
 {'speciesCode': 'grhowl',
  'comName': 'Great Horned Owl',
  'sciName': 'Bubo virginianus',
  'locId': 'L17717262',
  'locName': '8621 Black Valley Rd, Everett US-PA 39.97591, -78.41037MountainMeadowsBackyard',
  'obsDt': '2024-01-19 17:55',
  'howMany': 1,
  'lat': 39.975909,
  'lng': -78.410373,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S159307030'},
 {'speciesCode': 'tunswa',
  'comName': 'Tundra Swan',
  'sciName': 'Cygnus columbianus',
  'locId': 'L17717262',
  'locName': '8621 Black Valley Rd, Everett US-PA 39.97591, -78.41037MountainMeadowsBackyard',
  'obsDt': '2024-01-19 17:55',
  'howMany': 35,
  'lat': 39.975909,
  'lng

828

In [28]:
for item in data:
    print(item)

{'speciesCode': 'bnhcow', 'comName': 'Brown-headed Cowbird', 'sciName': 'Molothrus ater', 'locId': 'L11149270', 'locName': 'Habor Ridge Dr, Harrison US-OH 39.25857, -84.74789', 'obsDt': '2024-01-18 11:58', 'howMany': 2, 'lat': 39.258575, 'lng': -84.747889, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': True, 'subId': 'S159205336'}
{'speciesCode': 'perfal', 'comName': 'Peregrine Falcon', 'sciName': 'Falco peregrinus', 'locId': 'L3546352', 'locName': 'Carnegie Mellon University', 'obsDt': '2024-01-18 11:57', 'howMany': 1, 'lat': 40.4427503, 'lng': -79.9434328, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': False, 'subId': 'S159204987'}
{'speciesCode': 'daejun', 'comName': 'Dark-eyed Junco', 'sciName': 'Junco hyemalis', 'locId': 'L12198831', 'locName': 'Cayman Chemical', 'obsDt': '2024-01-18 11:56', 'howMany': 3, 'lat': 42.22805, 'lng': -83.722282, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': True, 'subId': 'S159204948'}
{'speciesCode': 'turvul', 'com

In [39]:
first_5 = data[1:5]
display(first_5)

[{'speciesCode': 'perfal',
  'comName': 'Peregrine Falcon',
  'sciName': 'Falco peregrinus',
  'locId': 'L3546352',
  'locName': 'Carnegie Mellon University',
  'obsDt': '2024-01-18 11:57',
  'howMany': 1,
  'lat': 40.4427503,
  'lng': -79.9434328,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': False,
  'subId': 'S159204987'},
 {'speciesCode': 'daejun',
  'comName': 'Dark-eyed Junco',
  'sciName': 'Junco hyemalis',
  'locId': 'L12198831',
  'locName': 'Cayman Chemical',
  'obsDt': '2024-01-18 11:56',
  'howMany': 3,
  'lat': 42.22805,
  'lng': -83.722282,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S159204948'},
 {'speciesCode': 'turvul',
  'comName': 'Turkey Vulture',
  'sciName': 'Cathartes aura',
  'locId': 'L14035235',
  'locName': "Whitley's Crossroads",
  'obsDt': '2024-01-18 11:56',
  'howMany': 3,
  'lat': 36.8728124,
  'lng': -76.77873,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S1

In [9]:
for item in data:
    print(item['howMany'])

10
3
12
3
6
1
32
1
1
1
1
3
1
10
1
8
1
2
3
1
3
1
1


KeyError: 'howMany'

In [32]:
dynamodb = boto3.resource('dynamodb')
table_name = 'RecentObservations'
table = dynamodb.Table(table_name)

dynamodb.Table(name='RecentObservations')


In [43]:
for item in first_5:
    table.put_item(
        TableName = table_name,
        Item = {
                'speciesCode': item['speciesCode'],
                'comName': item['comName'],
                'sciName': item['sciName'],
                'locId': item['locId'],
                'locName': item['locName'],
                'obsDt': item['obsDt'],
                'howMany': item['howMany'],
                'lat': str(item['lat']),
                'lng': str(item['lng']),
                'obsValid': item['obsValid'],
                'obsReviewed': item['obsReviewed'],
                'locationPrivate': item['locationPrivate'],
                'subId':  item['subId']
        }
    )

In [ ]:
for item in first_5:
    table.update_item(
        TableName = table_name,
        Key = {'speciesCode': item['speciesCode'], 'obsDt': item['obsDt']}
        Item = {
                'speciesCode': item['speciesCode'],
                'comName': item['comName'],
                'sciName': item['sciName'],
                'locId': item['locId'],
                'locName': item['locName'],
                'obsDt': item['obsDt'],
                'howMany': item['howMany'],
                'lat': str(item['lat']),
                'lng': str(item['lng']),
                'obsValid': item['obsValid'],
                'obsReviewed': item['obsReviewed'],
                'locationPrivate': item['locationPrivate'],
                'subId':  item['subId']
        }
    )

In [38]:
response = table.scan()
print(response)

{'Items': [{'locId': 'L14035235', 'locationPrivate': True, 'lng': Decimal('-76.77873'), 'speciesCode': 'horlar', 'sciName': 'Eremophila alpestris', 'obsReviewed': False, 'howMany': Decimal('1'), 'obsValid': True, 'locName': "Whitley's Crossroads", 'obsDt': '2024-01-18 11:56', 'comName': 'Horned Lark', 'subId': 'S159205287', 'lat': Decimal('36.8728124')}, {'locId': 'L3546352', 'locationPrivate': False, 'lng': Decimal('-79.9434328'), 'speciesCode': 'perfal', 'sciName': 'Falco peregrinus', 'obsReviewed': False, 'howMany': Decimal('1'), 'obsValid': True, 'locName': 'Carnegie Mellon University', 'obsDt': '2024-01-18 11:57', 'comName': 'Peregrine Falcon', 'subId': 'S159204987', 'lat': Decimal('40.4427503')}, {'locId': 'L12198831', 'locationPrivate': True, 'lng': Decimal('-83.722282'), 'speciesCode': 'daejun', 'sciName': 'Junco hyemalis', 'obsReviewed': False, 'howMany': Decimal('3'), 'obsValid': True, 'locName': 'Cayman Chemical', 'obsDt': '2024-01-18 11:56', 'comName': 'Dark-eyed Junco', 's

In [96]:
url = 'https://api.ebird.org/v2/ref/hotspot/US'
key = os.environ['EBIRD_API_KEY']
header = {'X-eBirdApiToken': key}
params = {'back': 30, 'includeProvisional': True, 'fmt':'json'}
response = requests.get(url, headers = header, params=params)


In [97]:
df = pd.DataFrame(response.json())
display(df)

,locId,locName,countryCode,subnational1Code,subnational2Code,lat,lng,latestObsDt,numSpeciesAllTime
0,L880996,'Akaka Falls State Park,US,US-HI,US-HI-001,19.851430,-155.152665,2024-01-15 10:24,52
1,L1652420,'Au'au Channel,US,US-HI,US-HI-009,20.834905,-156.740081,2024-01-18 09:40,46
2,L718307,**Back Bay NWR,US,US-VA,US-VA-810,36.672127,-75.915661,2024-01-18 08:48,333
3,L718378,**CHINCOTEAGUE NWR,US,US-VA,US-VA-001,37.910623,-75.347972,2024-01-17 09:16,344
4,L718525,**HUNTLEY MEADOWS PARK,US,US-VA,US-VA-059,38.757645,-77.098412,2024-01-18 15:50,249
...,...,...,...,...,...,...,...,...,...
54578,L28685617,"stakeout, Townsend’s Solitaire — Ivywood, Chis...",US,US-MN,US-MN-025,45.399266,-92.922174,2024-01-15 10:58,16
54579,L13137739,state land along river btwn 372 and 177 (acces...,US,US-CT,US-CT-003,41.674556,-72.884335,2023-12-31 13:56,32
54580,L9169407,wanakena,US,US-NY,US-NY-089,44.133202,-74.923536,2024-01-06 14:45,110
54581,L253053,ʻAiea Loop Trail and Keaīwa Heiau SP,US,US-HI,US-HI-003,21.398817,-157.900057,2024-01-18 17:30,44


In [98]:
unique_areas = pd.unique(df['subnational2Code'])
unique_areas

array(['US-HI-001', 'US-HI-009', 'US-VA-810', ..., 'US-TX-501',
       'US-OH-141', 'US-ND-051'], dtype=object)

In [100]:
dmv_areas = pd.unique(df['subnational2Code'].loc[df['subnational1Code'].isin(['US-VA', 'US-MD', 'US-DC'])])
dmv_areas
len(dmv_areas)

152

In [92]:
key = os.environ['EBIRD_API_KEY']
header = {'X-eBirdApiToken': key}
params = {'back': 1}

nat_data = []
for area in dmv_areas:
    url = f"https://api.ebird.org/v2/data/obs/{area}/recent"
    response = requests.get(url, headers = header, params=params)
    nat_data.extend(response.json())

In [93]:
nat_data

[{'speciesCode': 'ribgul',
  'comName': 'Ring-billed Gull',
  'sciName': 'Larus delawarensis',
  'locId': 'L246888',
  'locName': 'Lynnhaven Inlet',
  'obsDt': '2024-01-19 15:21',
  'howMany': 5,
  'lat': 36.9073525,
  'lng': -76.0918236,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': False,
  'subId': 'S159296187'},
 {'speciesCode': 'rebmer',
  'comName': 'Red-breasted Merganser',
  'sciName': 'Mergus serrator',
  'locId': 'L246888',
  'locName': 'Lynnhaven Inlet',
  'obsDt': '2024-01-19 15:21',
  'howMany': 1,
  'lat': 36.9073525,
  'lng': -76.0918236,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': False,
  'subId': 'S159296187'},
 {'speciesCode': 'buffle',
  'comName': 'Bufflehead',
  'sciName': 'Bucephala albeola',
  'locId': 'L246888',
  'locName': 'Lynnhaven Inlet',
  'obsDt': '2024-01-19 15:21',
  'howMany': 6,
  'lat': 36.9073525,
  'lng': -76.0918236,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': False,
  'subId': 'S159296

In [101]:
len(nat_data)

3247

In [105]:
url = 'https://api.ebird.org/v2/ref/region/list/subnational2/US-VA'
key = os.environ['EBIRD_API_KEY']
header = {'X-eBirdApiToken': key}
params = {'back': 30, 'includeProvisional': True, 'fmt':'json'}
response = requests.get(url, headers = header, params=params)
len(response.json())


133

In [110]:
url = 'https://api.ebird.org/v2/ref/region/list/subnational2/US-MD'
key = os.environ['EBIRD_API_KEY']
header = {'X-eBirdApiToken': key}
params = {'back': 30, 'includeProvisional': True, 'fmt':'json'}
response = requests.get(url, headers = header, params=params)
print(response.json())
len(response.json())

[{'code': 'US-MD-001', 'name': 'Allegany'}, {'code': 'US-MD-003', 'name': 'Anne Arundel'}, {'code': 'US-MD-005', 'name': 'Baltimore'}, {'code': 'US-MD-009', 'name': 'Calvert'}, {'code': 'US-MD-011', 'name': 'Caroline'}, {'code': 'US-MD-013', 'name': 'Carroll'}, {'code': 'US-MD-015', 'name': 'Cecil'}, {'code': 'US-MD-017', 'name': 'Charles'}, {'code': 'US-MD-019', 'name': 'Dorchester'}, {'code': 'US-MD-021', 'name': 'Frederick'}, {'code': 'US-MD-023', 'name': 'Garrett'}, {'code': 'US-MD-025', 'name': 'Harford'}, {'code': 'US-MD-027', 'name': 'Howard'}, {'code': 'US-MD-029', 'name': 'Kent'}, {'code': 'US-MD-031', 'name': 'Montgomery'}, {'code': 'US-MD-033', 'name': "Prince George's"}, {'code': 'US-MD-035', 'name': "Queen Anne's"}, {'code': 'US-MD-039', 'name': 'Somerset'}, {'code': 'US-MD-037', 'name': "St. Mary's"}, {'code': 'US-MD-041', 'name': 'Talbot'}, {'code': 'US-MD-043', 'name': 'Washington'}, {'code': 'US-MD-045', 'name': 'Wicomico'}, {'code': 'US-MD-047', 'name': 'Worcester'}]


23

In [114]:
codes = [item['code'] for item in response.json()]
print(codes)

['US-MD-001', 'US-MD-003', 'US-MD-005', 'US-MD-009', 'US-MD-011', 'US-MD-013', 'US-MD-015', 'US-MD-017', 'US-MD-019', 'US-MD-021', 'US-MD-023', 'US-MD-025', 'US-MD-027', 'US-MD-029', 'US-MD-031', 'US-MD-033', 'US-MD-035', 'US-MD-039', 'US-MD-037', 'US-MD-041', 'US-MD-043', 'US-MD-045', 'US-MD-047']


In [109]:
url = 'https://api.ebird.org/v2/ref/region/list/subnational2/US-DC'
key = os.environ['EBIRD_API_KEY']
header = {'X-eBirdApiToken': key}
params = {'back': 30, 'includeProvisional': True, 'fmt':'json'}
response = requests.get(url, headers = header, params=params)
print(response.json())
len(response.json())

[{'code': 'US-DC-001', 'name': 'District of Columbia'}]


1

In [7]:
def get_regions(region_type: str, parent_codes: list) -> list:
    """This function returns a list of eBird regions for a specified region type (country, subnational1, subnational2)
       and a list of parent region codes (e.g., 'US' or 'US-VA')"""
    
    key = os.environ['EBIRD_API_KEY']
    header = {'X-eBirdApiToken': key}
    params = {'fmt':'json'}
    region_codes = []
    for code in parent_codes:
        url = f"https://api.ebird.org/v2/ref/region/list/{region_type}/{code}"
        response = requests.get(url, headers = header, params=params)
        sub_codes = [item['code'] for item in response.json()]
        region_codes.extend(sub_codes)
    
    return region_codes

In [121]:
get_regions('subnational2', ['US-VA', 'US-MD', 'US-DC'])

['US-VA-001',
 'US-VA-003',
 'US-VA-510',
 'US-VA-005',
 'US-VA-007',
 'US-VA-009',
 'US-VA-011',
 'US-VA-013',
 'US-VA-015',
 'US-VA-017',
 'US-VA-019',
 'US-VA-021',
 'US-VA-023',
 'US-VA-520',
 'US-VA-025',
 'US-VA-027',
 'US-VA-029',
 'US-VA-530',
 'US-VA-031',
 'US-VA-033',
 'US-VA-035',
 'US-VA-036',
 'US-VA-037',
 'US-VA-540',
 'US-VA-550',
 'US-VA-041',
 'US-VA-043',
 'US-VA-570',
 'US-VA-580',
 'US-VA-045',
 'US-VA-047',
 'US-VA-049',
 'US-VA-590',
 'US-VA-051',
 'US-VA-053',
 'US-VA-595',
 'US-VA-057',
 'US-VA-059',
 'US-VA-600',
 'US-VA-610',
 'US-VA-061',
 'US-VA-063',
 'US-VA-065',
 'US-VA-067',
 'US-VA-620',
 'US-VA-069',
 'US-VA-630',
 'US-VA-640',
 'US-VA-071',
 'US-VA-073',
 'US-VA-075',
 'US-VA-077',
 'US-VA-079',
 'US-VA-081',
 'US-VA-083',
 'US-VA-650',
 'US-VA-085',
 'US-VA-660',
 'US-VA-087',
 'US-VA-089',
 'US-VA-091',
 'US-VA-670',
 'US-VA-093',
 'US-VA-095',
 'US-VA-099',
 'US-VA-101',
 'US-VA-097',
 'US-VA-103',
 'US-VA-105',
 'US-VA-678',
 'US-VA-107',
 'US-V

In [5]:
import concurrent.futures
import requests

def get_recent_obs(days: int, region_codes: list) -> list:
    """Function to get the most recent observations for a set of regions going back a number of specified days"""
    key = os.environ['EBIRD_API_KEY']
    headers = {'X-eBirdApiToken': key}
    params = {'back': days}  # Specifies number of days to retrieve data for

    # Use a session for connection pooling
    with requests.Session() as session:
        session.headers.update(headers)

        def fetch_data(code):
            url = f"https://api.ebird.org/v2/data/obs/{code}/recent"
            response = session.get(url, params=params)
            return response.json()

        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = executor.map(fetch_data, region_codes)

        data = [entry for result in results for entry in result]
        print(data)
        return data


In [8]:
data = get_recent_obs(1,get_regions('subnational2', ['US-VA', 'US-MD', 'US-DC']))

[{'speciesCode': 'amecro', 'comName': 'American Crow', 'sciName': 'Corvus brachyrhynchos', 'locId': 'L17299844', 'locName': '16443 Hopeland Way, Bloxom US-VA (37.808570, -75.540898)', 'obsDt': '2024-01-20 07:51', 'howMany': 2, 'lat': 37.80871, 'lng': -75.540932, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': True, 'subId': 'S159345594'}, {'speciesCode': 'baleag', 'comName': 'Bald Eagle', 'sciName': 'Haliaeetus leucocephalus', 'locId': 'L17299844', 'locName': '16443 Hopeland Way, Bloxom US-VA (37.808570, -75.540898)', 'obsDt': '2024-01-20 07:51', 'howMany': 1, 'lat': 37.80871, 'lng': -75.540932, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': True, 'subId': 'S159345594'}, {'speciesCode': 'blkvul', 'comName': 'Black Vulture', 'sciName': 'Coragyps atratus', 'locId': 'L17299844', 'locName': '16443 Hopeland Way, Bloxom US-VA (37.808570, -75.540898)', 'obsDt': '2024-01-20 07:51', 'howMany': 1, 'lat': 37.80871, 'lng': -75.540932, 'obsValid': True, 'obsReviewed': False, 

In [16]:
import time

def add_ttl(data):
    """This function adds a time to live attribute to each observation based on the observation date + 30 days"""
    data_with_ttl = []
    for entry in data:
        obs_timestamp = time.mktime(time.strptime(entry['obsDt'], "%Y-%m-%d %H:%M"))
        ttl_timestamp = int(obs_timestamp + (30*24*60*60)) #Add 30 days in seconds. TTL is in epoch time. 
        entry['ttl'] = ttl_timestamp

        data_with_ttl.append(entry)
    
    print(data_with_ttl)
    return data_with_ttl



In [19]:
add_ttl(data)

[{'speciesCode': 'amecro', 'comName': 'American Crow', 'sciName': 'Corvus brachyrhynchos', 'locId': 'L17299844', 'locName': '16443 Hopeland Way, Bloxom US-VA (37.808570, -75.540898)', 'obsDt': '2024-01-20 07:51', 'howMany': 2, 'lat': 37.80871, 'lng': -75.540932, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': True, 'subId': 'S159345594', 'ttl': 1708347060}, {'speciesCode': 'baleag', 'comName': 'Bald Eagle', 'sciName': 'Haliaeetus leucocephalus', 'locId': 'L17299844', 'locName': '16443 Hopeland Way, Bloxom US-VA (37.808570, -75.540898)', 'obsDt': '2024-01-20 07:51', 'howMany': 1, 'lat': 37.80871, 'lng': -75.540932, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': True, 'subId': 'S159345594', 'ttl': 1708347060}, {'speciesCode': 'blkvul', 'comName': 'Black Vulture', 'sciName': 'Coragyps atratus', 'locId': 'L17299844', 'locName': '16443 Hopeland Way, Bloxom US-VA (37.808570, -75.540898)', 'obsDt': '2024-01-20 07:51', 'howMany': 1, 'lat': 37.80871, 'lng': -75.540932, 'o

[{'speciesCode': 'amecro',
  'comName': 'American Crow',
  'sciName': 'Corvus brachyrhynchos',
  'locId': 'L17299844',
  'locName': '16443 Hopeland Way, Bloxom US-VA (37.808570, -75.540898)',
  'obsDt': '2024-01-20 07:51',
  'howMany': 2,
  'lat': 37.80871,
  'lng': -75.540932,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S159345594',
  'ttl': 1708347060},
 {'speciesCode': 'baleag',
  'comName': 'Bald Eagle',
  'sciName': 'Haliaeetus leucocephalus',
  'locId': 'L17299844',
  'locName': '16443 Hopeland Way, Bloxom US-VA (37.808570, -75.540898)',
  'obsDt': '2024-01-20 07:51',
  'howMany': 1,
  'lat': 37.80871,
  'lng': -75.540932,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S159345594',
  'ttl': 1708347060},
 {'speciesCode': 'blkvul',
  'comName': 'Black Vulture',
  'sciName': 'Coragyps atratus',
  'locId': 'L17299844',
  'locName': '16443 Hopeland Way, Bloxom US-VA (37.808570, -75.540898)',
  'obsDt': '2024-01